# Feature Engineering - Tips (Versão Spark)

**Conversão Pandas → Spark**

Este notebook:
1. Lê tips da camada Bronze (Spark)
2. Filtra apenas tips de business válidos (Silver)
3. Cria features de usuários (user_tip_count)
4. Cria features de business:
   - tip_count (contagem)
   - recency_score (baseado na data do último tip)
5. Aplica normalização (log + MinMaxScaler)
6. Salva na Silver layer

**Vantagens vs Pandas:**
- ✅ 10x mais rápido
- ✅ Sem problemas de memória
- ✅ Acessa datalake diretamente

---

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, max as spark_max, log1p, 
    to_timestamp, current_timestamp, datediff, lit, broadcast
)
from pyspark.ml.feature import MinMaxScaler, VectorAssembler
from pyspark.sql.types import DoubleType
import pyspark.sql.functions as F

In [2]:
# ⚡ CONFIGURAÇÃO SPARK OTIMIZADA
spark = SparkSession.builder \
    .appName("Tip Feature Engineering") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

print(f"✅ Spark version: {spark.version}")

✅ Spark version: 3.5.0


In [3]:
# Configuração de Paths
BASE_PATH = '/home/jovyan/work'
DATA_PATH = f'{BASE_PATH}/data'
BRONZE_PATH = f'{DATA_PATH}/bronze'
SILVER_PATH = f'{DATA_PATH}/silver'

print(f"🥉 Bronze: {BRONZE_PATH}")
print(f"🥈 Silver: {SILVER_PATH}")

🥉 Bronze: /home/jovyan/work/data/bronze
🥈 Silver: /home/jovyan/work/data/silver


## 1. Carregar Dados do Datalake

In [4]:
print("\n📥 [1/5] Carregando dados do datalake...\n")

# 1. Tips (Bronze)
print("   📦 Carregando tips...")
df_tips = spark.read.parquet(f"{BRONZE_PATH}/tip")
count_tips_total = df_tips.count()
print(f"      ✅ Tips: {count_tips_total:,} registros")

# 2. Business (Silver) - Já filtrado
print("   📦 Carregando business...")
df_business = spark.read.parquet(f"{SILVER_PATH}/business")
count_business = df_business.count()
print(f"      ✅ Business: {count_business:,} registros")

print("\n✅ Dados carregados!")


📥 [1/5] Carregando dados do datalake...

   📦 Carregando tips...
      ✅ Tips: 908,915 registros
   📦 Carregando business...
      ✅ Business: 64,645 registros

✅ Dados carregados!


In [6]:
# Verificar schema dos tips
print("📋 Schema do Tip:\n")
df_tips.printSchema()

print("\n📊 Amostra:")
df_tips.show(5, truncate=5)

📋 Schema do Tip:

root
 |-- business_id: string (nullable = true)
 |-- compliment_count: long (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)


📊 Amostra:
+-----------+----------------+-----+-----+-------+
|business_id|compliment_count| date| text|user_id|
+-----------+----------------+-----+-----+-------+
|      Ea...|               0|20...|Ok...|  Wp...|
|      1_...|               0|20...|Av...|  SL...|
|      wE...|               0|20...|Al...|  mM...|
|      JL...|               0|20...|Th...|  Th...|
|      un...|               0|20...|Gr...|  xt...|
+-----------+----------------+-----+-----+-------+
only showing top 5 rows



## 2. Filtrar Tips por Business Válidos

In [7]:
print("\n🔧 [2/5] Filtrando tips por business válidos...\n")

# Broadcast join (business IDs é pequeno)
valid_business_ids = df_business.select('business_id')

df_tips_filtered = df_tips.join(
    broadcast(valid_business_ids),
    on='business_id',
    how='inner'
)

# Persist - vai ser usado múltiplas vezes
df_tips_filtered.persist()

count_tips_filtered = df_tips_filtered.count()

print(f"   📊 Tips originais: {count_tips_total:,}")
print(f"   📊 Tips filtrados: {count_tips_filtered:,}")
print(f"   📉 Redução: {100 * (1 - count_tips_filtered/count_tips_total):.1f}%")
print(f"\n   ✅ Filtrado!")


🔧 [2/5] Filtrando tips por business válidos...

   📊 Tips originais: 908,915
   📊 Tips filtrados: 471,886
   📉 Redução: 48.1%

   ✅ Filtrado!


In [12]:
df_tips_filtered.show()

+--------------------+----------------+-------------------+--------------------+--------------------+
|         business_id|compliment_count|               date|                text|             user_id|
+--------------------+----------------+-------------------+--------------------+--------------------+
|EagkHaaC-kUozD3MP...|               0|2012-08-25 15:45:59|Okay totally get ...|WpCNsz24sa8WKPNjO...|
|wEdzUMaLE2ebYoe7Z...|               0|2015-07-29 03:44:09|Always epic :-) b...|mMbyHtXB0Spjs6HtB...|
|JLixvCikc5JYGcnva...|               0|2017-12-23 04:37:35|This place is ama...|ThxOrJ0-guAiHEk_H...|
|aUoMG97DMJG4nmwhT...|               0|2012-02-03 20:45:27|Lunch.... Jumbo l...|iTDRa5mQ0iW6RKifV...|
|fdmLML9X_0mWWG3TF...|               0|2010-11-12 00:07:10|Breakfast sampler...|RdGX12xRj4tYHSBrr...|
|YaEwp8emNzySe-d0M...|               0|2012-07-07 18:07:33|   Get the Jae Yook!|snDMqT-3QMWSP-ngv...|
|M-oi_kkI0AhbR-xqR...|               0|2014-05-24 01:08:13|The worst bbq I h...|rY

## 3. Features de Usuários (User Side)

In [8]:
df_user_features = df_tips_filtered \
    .groupBy('user_id') \
    .agg(
        count('business_id').alias('user_tip_count')
    )

count_users = df_user_features.count()
print(f"   📊 Total de usuários com tips: {count_users:,}")


print("\n   📈 Distribuição de tips por usuário:")
df_user_features.select('user_tip_count').summary().show()

print("   ✅ Features de usuários geradas!")


👤 [3/5] Gerando features de usuários...

   📊 Total de usuários com tips: 186,363

   📈 Distribuição de tips por usuário:
+-------+------------------+
|summary|    user_tip_count|
+-------+------------------+
|  count|            186363|
|   mean|2.5320798656385657|
| stddev|  9.62150408823629|
|    min|                 1|
|    25%|                 1|
|    50%|                 1|
|    75%|                 2|
|    max|              1360|
+-------+------------------+

   ✅ Features de usuários geradas!


+--------------------+--------------+
|             user_id|user_tip_count|
+--------------------+--------------+
|8DYVfcMbskCpm3tZl...|             6|
|gIOJu-di8dTiSlRYc...|             7|
|kmFCBdDDwnszzPjSM...|             3|
|9mEStNO5uorGiECK6...|             1|
|yvMtlZnDm3cdLSdjx...|             2|
|4Oysk2ZYl52ID-7xs...|             1|
|eOPh_nXTW4l_vk9j8...|             1|
|Ha5bMb0SfRcvX7CS0...|             1|
|KnzMF1NoBRqtJULZ0...|             1|
|cOS-MNurk0vTQf2QL...|             3|
|8_dUX7813Xuk4z_pQ...|             1|
|esji5clHjFmS-tU6_...|             2|
|ZsHK3rZszbTUF_Fkw...|             1|
|c8otGYB12t1mnR-VR...|             1|
|1K6DwmdsX7NUgZmbh...|             1|
|48rgWyQ4TjIPLDavO...|             1|
|Wx7PqffTfBmSRneSo...|             2|
|V9VDc3h0VHyH3yVQd...|             1|
|tCmgAsZptUbj5IpbP...|             1|
|HVzcdZlpUkXWYshwF...|             1|
+--------------------+--------------+
only showing top 20 rows



In [9]:
# Salvar User Features
import os
import shutil

user_output_path = f'{SILVER_PATH}/tip_features_user'

# Remove se existir
if os.path.exists(user_output_path):
    shutil.rmtree(user_output_path)

df_user_features \
    .repartition(5) \
    .write \
    .mode('overwrite') \
    .option('compression', 'snappy') \
    .parquet(user_output_path)

print(f"\n   💾 Salvo: {user_output_path}")
print(f"      Total: {count_users:,} usuários")


   💾 Salvo: /home/jovyan/work/data/silver/tip_features_user
      Total: 186,363 usuários


## 4. Features de Business (Item Side)

In [13]:
print("\n🏪 [4/5] Gerando features de business...\n")

# Converter date para timestamp se necessário
df_tips_with_date = df_tips_filtered.withColumn(
    'date_timestamp',
    to_timestamp(col('date'), 'yyyy-MM-dd HH:mm:ss')
)

# Agregação por business
df_business_features = df_tips_with_date \
    .groupBy('business_id') \
    .agg(
        count('user_id').alias('tip_count'),
        spark_max('date_timestamp').alias('last_tip_date')
    )

# Calcular dias desde o último tip
df_business_features = df_business_features.withColumn(
    'days_since_last_tip',
    datediff(current_timestamp(), col('last_tip_date'))
)

count_businesses = df_business_features.count()
print(f"   📊 Business com tips: {count_businesses:,}")

# Estatísticas
print("\n   📈 Distribuição de tips por business:")
df_business_features.select('tip_count', 'days_since_last_tip').summary().show()

print("   ✅ Features de business geradas!")


🏪 [4/5] Gerando features de business...

   📊 Business com tips: 44,904

   📈 Distribuição de tips por business:
+-------+------------------+-------------------+
|summary|         tip_count|days_since_last_tip|
+-------+------------------+-------------------+
|  count|             44904|              44904|
|   mean| 10.50877427400677|  2721.316742383752|
| stddev|28.974293196464423| 1000.1498461319642|
|    min|                 1|               1413|
|    25%|                 1|               1818|
|    50%|                 3|               2538|
|    75%|                 8|               3408|
|    max|              2571|               5986|
+-------+------------------+-------------------+

   ✅ Features de business geradas!


## 5. Engenharia Matemática (Log + Normalização)

In [14]:
print("\n🧪 [5/5] Aplicando transformações matemáticas...\n")

# A. Log na Popularidade (tip_count)
print("   📐 Aplicando log1p no tip_count...")
df_features = df_business_features.withColumn(
    'tip_count_log',
    log1p(col('tip_count'))
)

# B. Recency Score (inverso dos dias)
# Score = 1 / (dias + 7)
# +7 para suavizar (evitar divisão por 0 e dar peso aos muito recentes)
print("   📐 Calculando recency score...")
df_features = df_features.withColumn(
    'recency_score',
    1.0 / (col('days_since_last_tip') + 7)
)

print("\n   ✅ Transformações aplicadas!")


🧪 [5/5] Aplicando transformações matemáticas...

   📐 Aplicando log1p no tip_count...
   📐 Calculando recency score...

   ✅ Transformações aplicadas!


In [15]:
print("\n📊 Aplicando MinMaxScaler...\n")

# Colunas para normalizar
cols_to_normalize = ['tip_count_log', 'recency_score']

# Aplicar MinMaxScaler em cada coluna
for col_name in cols_to_normalize:
    print(f"   ⚙️  Normalizando {col_name}...")
    
    # 1. Criar Vector
    assembler = VectorAssembler(
        inputCols=[col_name],
        outputCol=f"{col_name}_vec"
    )
    df_features = assembler.transform(df_features)
    
    # 2. Aplicar MinMaxScaler
    scaler = MinMaxScaler(
        inputCol=f"{col_name}_vec",
        outputCol=f"{col_name}_scaled"
    )
    scaler_model = scaler.fit(df_features)
    df_features = scaler_model.transform(df_features)
    
    # 3. Extrair valor do Vector
    from pyspark.sql.functions import udf
    
    def extract_value(vector):
        if vector is None:
            return 0.0
        return float(vector[0])
    
    extract_udf = udf(extract_value, DoubleType())
    
    df_features = df_features.withColumn(
        col_name,
        extract_udf(col(f"{col_name}_scaled"))
    )
    
    # 4. Dropar colunas temporárias
    df_features = df_features.drop(f"{col_name}_vec", f"{col_name}_scaled")

print("\n   ✅ MinMaxScaler aplicado!")


📊 Aplicando MinMaxScaler...

   ⚙️  Normalizando tip_count_log...
   ⚙️  Normalizando recency_score...

   ✅ MinMaxScaler aplicado!


In [16]:
# Selecionar colunas finais
df_final = df_features.select(
    'business_id',
    'tip_count_log',
    'recency_score'
)

print("\n📊 Features finais:\n")
df_final.show(10)

# Estatísticas das features normalizadas
print("\n📈 Estatísticas das features normalizadas (devem estar entre 0-1):")
df_final.select('tip_count_log', 'recency_score').summary().show()


📊 Features finais:

+--------------------+-------------------+-------------------+
|         business_id|      tip_count_log|      recency_score|
+--------------------+-------------------+-------------------+
|PNby7mawC0ecfg-uE...| 0.2381168577713732| 0.9384320280843057|
|P2XJbQZmf1zvWp9L_...|0.39573932474489987| 0.9097675952579826|
|YdZS3QkpjgHU2zIJ_...| 0.3782567098971756| 0.6541971000475326|
|PQYuPr2Dfm2S49EUL...|  0.504368024193075| 0.6910618958048563|
|rQyJXOiZ39eRJ2l9O...| 0.3582685818052369| 0.8297611084670037|
|bWgi0dQSRbTa44YZS...| 0.2381168577713732| 0.6345990226601098|
|0wZJkj-OnZ7Pmubls...| 0.5491919710788076| 0.9844963045171038|
|BJOGo_upuBElDT_xO...|0.46035793900172334| 0.9375943686024595|
|2IahpaBR4U2Kdy9HF...| 0.5622555616202339|0.49126370027020144|
|-f5A6KKUu4jZN1qpt...|0.09681784033634044| 0.9460218677783597|
+--------------------+-------------------+-------------------+
only showing top 10 rows


📈 Estatísticas das features normalizadas (devem estar entre 0-1):
+---

In [18]:
df_final.count()


44904

## 6. Salvar na Silver Layer

In [19]:
print("\n💾 Salvando business features na Silver Layer...\n")

business_output_path = f'{SILVER_PATH}/tip_features_business'

# Remove se existir
if os.path.exists(business_output_path):
    shutil.rmtree(business_output_path)
    print(f"   🗑️  Removido arquivo antigo: {business_output_path}")

# Salvar
df_final \
    .repartition(5) \
    .write \
    .mode('overwrite') \
    .option('compression', 'snappy') \
    .parquet(business_output_path)

print(f"\n{'='*60}")
print(f"✅ BUSINESS FEATURES SALVAS: {business_output_path}")
print(f"   📊 Total: {count_businesses:,} business")
print(f"   📦 Partições: 5")
print(f"   🗜️  Compressão: SNAPPY")
print(f"{'='*60}")


💾 Salvando business features na Silver Layer...


✅ BUSINESS FEATURES SALVAS: /home/jovyan/work/data/silver/tip_features_business
   📊 Total: 44,904 business
   📦 Partições: 5
   🗜️  Compressão: SNAPPY


In [20]:
# Verificar arquivos salvos
print("\n🔍 Verificando arquivos salvos...\n")

# User features
df_user_verify = spark.read.parquet(user_output_path)
print(f"✅ User features: {df_user_verify.count():,} registros")
print("   Colunas:", df_user_verify.columns)

# Business features
df_business_verify = spark.read.parquet(business_output_path)
print(f"\n✅ Business features: {df_business_verify.count():,} registros")
print("   Colunas:", df_business_verify.columns)

print("\n📊 Amostra do resultado final:")
df_business_verify.show(5)


🔍 Verificando arquivos salvos...

✅ User features: 186,363 registros
   Colunas: ['user_id', 'user_tip_count']

✅ Business features: 44,904 registros
   Colunas: ['business_id', 'tip_count_log', 'recency_score']

📊 Amostra do resultado final:
+--------------------+-------------------+------------------+
|         business_id|      tip_count_log|     recency_score|
+--------------------+-------------------+------------------+
|u0wF_sqDWzJ3oVq1I...|0.17498420027487216|0.4317776833220361|
|T_pj13mZUV_2YirOF...|                0.0| 0.750448632484408|
|vOe2KAgnb9dA6Np66...|                0.0|0.6825105177459176|
|SG1GdORwoqxl_WEAJ...|                0.0|0.5038957136383712|
|_Q56kbKTbYW6dT2HC...| 0.2989214844085594|0.2501734936720353|
+--------------------+-------------------+------------------+
only showing top 5 rows



In [21]:
# Liberar memória
print("\n🧹 Limpando cache...")
df_tips_filtered.unpersist()
spark.catalog.clearCache()
print("✅ Cache limpo!")

print("\n" + "="*60)
print("🎉 PROCESSAMENTO COMPLETO!")
print("="*60)
print("\n📁 Arquivos gerados:")
print(f"   1. {user_output_path}")
print(f"   2. {business_output_path}")
print("\n🔄 Próximo passo:")
print("   Execute o notebook: 03_checkin_feature_engineering_SPARK.ipynb")
print("="*60)


🧹 Limpando cache...
✅ Cache limpo!

🎉 PROCESSAMENTO COMPLETO!

📁 Arquivos gerados:
   1. /home/jovyan/work/data/silver/tip_features_user
   2. /home/jovyan/work/data/silver/tip_features_business

🔄 Próximo passo:
   Execute o notebook: 03_checkin_feature_engineering_SPARK.ipynb


---

## 📊 Resumo do Pipeline

**Input:**
- `bronze/tip` - Tips brutos (~900K)
- `silver/business` - Business filtrados (~64K)

**Processamento:**
1. ✅ Filtrar tips de business válidos (~471K tips)
2. ✅ Agregar por user_id → user_tip_count
3. ✅ Agregar por business_id:
   - tip_count (contagem)
   - last_tip_date (data mais recente)
   - days_since_last_tip (dias desde último)
4. ✅ Transformações:
   - Log(tip_count)
   - Recency score = 1/(dias+7)
   - MinMaxScaler [0-1]

**Output:**
- `silver/tip_features_user` - Features por usuário
- `silver/tip_features_business` - Features por business

**Features Finais (Business):**
- `business_id` - ID único
- `tip_count_log` - Popularidade normalizada [0-1]
- `recency_score` - Atividade recente normalizada [0-1]

**Performance:**
- Pandas: 5-10 minutos
- Spark: 1-2 minutos ⚡
- **Speedup: ~5-10x mais rápido!**

---